In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import yfinance as yf
from torch.utils.data import Dataset, DataLoader
from cvxpylayers.torch import CvxpyLayer
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
import torch.autograd as autograd
from itertools import accumulate

In [2]:
# Dataset creation
def download_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    returns = data.pct_change().dropna()
    return returns

In [3]:
class PortfolioDataset(Dataset):
    def __init__(self, returns_data, subseries_len=128):
        self.returns_data = returns_data
        self.subseries_len = subseries_len
        self.mu_tags = []
        self.sigma_tags = []

        for i in range(len(returns_data) - subseries_len):
            subseries = returns_data[i:i + subseries_len]
            sample_mu = subseries.mean(dim=0)
            sample_sigma = torch.cov(subseries.T)
            self.mu_tags.append(sample_mu)
            self.sigma_tags.append(sample_sigma)

    def __len__(self):
        return len(self.returns_data) - self.subseries_len

    def __getitem__(self, idx):
        subseries = self.returns_data[idx:idx + self.subseries_len]
        mu_tag = self.mu_tags[idx]
        sigma_tag = self.sigma_tags[idx]
        return subseries, mu_tag, sigma_tag

In [4]:
# Model definition
class PortfolioNet(nn.Module):
    def __init__(self, input_dim, n_assets):
        super(PortfolioNet, self).__init__()
        self.fc_mu = nn.Linear(input_dim, n_assets)
        self.fc_L = nn.Linear(input_dim, n_assets * (n_assets + 1) // 2)

    def forward(self, x):
        mu = self.fc_mu(x.reshape(x.shape[0], -1))
        L_flat = self.fc_L(x.reshape(x.shape[0], -1))

        indices = torch.tril_indices(x.shape[2], x.shape[2], offset=0)
        L = torch.zeros(x.shape[0], x.shape[2], x.shape[2])
        for i in range(x.shape[0]):
            L[i, indices[0], indices[1]] = L_flat[i]

        Sigma = torch.matmul(L, L.transpose(1, 2))

        return mu, Sigma, L

In [5]:
class EnvelopeFunction(autograd.Function):
    @staticmethod
    def forward(ctx, variables, parameters, objective, inequalities, equalities, cvxpy_opts, *batch_params):
        cp_inequalities = [ineq(*variables, *parameters) <= 0 for ineq in inequalities]
        cp_equalities = [eq(*variables, *parameters) == 0 for eq in equalities]
        problem = cp.Problem(
            cp.Minimize(objective(*variables, *parameters)),
            cp_inequalities + cp_equalities
        )
        
        outputs = []
        gradients = []
        
        for batch in range(batch_params[0].shape[0]):
            with torch.no_grad():
                params = [p[batch] for p in batch_params]
                # Solve optimization problem
                for i, p in enumerate(parameters):
                    p.value = params[i].double().numpy()
                problem.solve(**cvxpy_opts)

                # Retrieve variables and dual variables
                z = [torch.tensor(v.value, dtype=params[0].dtype, device=params[0].device) for v in variables]
                lam = [torch.tensor(c.dual_value, dtype=params[0].dtype, device=params[0].device)
                       for c in cp_inequalities]
                nu = [torch.tensor(c.dual_value, dtype=params[0].dtype, device=params[0].device)
                      for c in cp_equalities]
                optimal_val = torch.tensor(problem.value, dtype=params[0].dtype, device=params[0].device)
            
            with torch.enable_grad():
                params = [p[batch].detach().requires_grad_(True) for p in batch_params]

                g = [ineq(*z, *params) for ineq in inequalities]
                h = [eq(*z, *params) for eq in equalities]                
                L = (objective(*z, *params) +
                     sum((u * v).sum() for u, v in zip(lam, g)) +
                     sum((u * v).sum() for u, v in zip(nu, h)))
            
                # Compute derivatives using the envelope theorem
                dproblem = autograd.grad(L, params, create_graph=True)
            
            # Save outputs and gradients for each batch
            outputs.append(optimal_val)
            gradients.append(dproblem)
    
        # Save outputs and gradients for backward pass
        ctx.save_for_backward(*batch_params)
        ctx.gradients = gradients
        return torch.stack(outputs)


    @staticmethod
    def backward(ctx, grad_output):
        batch_params = ctx.saved_tensors
        gradients = ctx.gradients

        # Use precomputed gradients for each batch
        batch_size = len(gradients)
        grad_inputs = [None] * len(batch_params)
        for i in range(len(batch_params)):
            grad_inputs[i] = torch.stack([gradients[batch][i] * grad_output[batch] for batch in range(batch_size)])
        
        return (None, None, None, None, None, None, *grad_inputs)  # Return None for non-tensor inputs


class EnvelopeLayer(nn.Module):
    def __init__(self, variables, parameters, objective, inequalities, equalities, **cvxpy_opts):
        super().__init__()
        self.variables = variables
        self.parameters = parameters
        self.objective = objective
        self.inequalities = inequalities
        self.equalities = equalities
        self.cvxpy_opts = cvxpy_opts

    def forward(self, *batch_params):
        return EnvelopeFunction.apply(
            self.variables,
            self.parameters,
            self.objective,
            self.inequalities,
            self.equalities,
            self.cvxpy_opts,
            *batch_params
        )

In [6]:
def create_cvxpy_layer(n, lambda_):
    x = cp.Variable(n)
    P_sqrt = cp.Parameter((n, n))
    mu_param = cp.Parameter(n)

    quad_form = cp.sum_squares(P_sqrt @ x)
    objective = cp.Minimize(0.5 * quad_form - lambda_ * mu_param.T @ x)
    constraints = [cp.sum(x) == 1, x >= 0]
    problem = cp.Problem(objective, constraints)
    assert problem.is_dpp(), "Problem is not DPP compliant."
    cvxpylayer = CvxpyLayer(problem, parameters=[mu_param, P_sqrt], variables=[x])
    return cvxpylayer

In [7]:
# Training step
np.random.seed(42)
torch.manual_seed(42)

tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA',  # Technology
    'PG', 'KO', 'PEP', 'WMT', 'COST',  # Consumer Goods
    'JNJ', 'PFE', 'MRK', 'ABT', 'AMGN',  # Healthcare
    'JPM', 'BAC', 'WFC', 'C', 'GS',  # Financials
    'XOM', 'CVX', 'COP', 'SLB', 'OXY',  # Energy
    'BA', 'CAT', 'UPS', 'DE', 'MMM',  # Industrials
    'DIS', 'TGT', 'MCD', 'LVMUY', 'NFLX',  # Consumer Services
    'LIN', 'NEM', 'CL', 'VMC', 'FCX',  # Materials
    'AMT', 'PLD', 'SPG', 'O', 'EQIX',  # Real Estate
    'NEE', 'DUK', 'XEL', 'SO', 'EXC'  # Utilities
]
returns = download_data(tickers, start_date='2015-01-01', end_date='2024-01-01')
returns = returns.values

returns_tensor = torch.tensor(returns, dtype=torch.float32)

subseries_len = 28

dataset = PortfolioDataset(returns_tensor, subseries_len=subseries_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

n_assets = len(tickers)
input_dim = n_assets * subseries_len

model = PortfolioNet(input_dim=input_dim, n_assets=n_assets)

optimizer = optim.Adam(model.parameters(), lr=0.001)

lambda_, alpha, beta, gamma = 0.5, 0, 0, 1  # Weights mu loss, Sigma loss, portfolio loss 

z = cp.Variable(n_assets)
L = cp.Parameter((n_assets, n_assets))
mu = cp.Parameter(n_assets)

def f_(z, L, mu):
    return 0.5 * cp.sum_squares(L @ z) - lambda_*mu.T @ z if isinstance(z, cp.Variable) else 0.5*torch.sum((L @ z)**2) - lambda_*mu@z
def g_(z, L, mu):
    return -z
def h_(z, L, mu):
    return cp.sum(z) - 1 if isinstance(z, cp.Variable) else torch.sum(z) - 1

envelope_layer = EnvelopeLayer(variables=[z], parameters=[L, mu], 
         objective=f_, inequalities=[g_], equalities=[h_])

num_epochs = 5
memory_stored = 0
for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{num_epochs}", position=0)
    for _, (features_batch, mu_tags_batch, sigma_tags_batch) in progress_bar:
        optimizer.zero_grad()
        init_memory = torch.cuda.memory_allocated()
        
        mu_hat, Sigma_hat, L_hat = model(features_batch)
        
        mu_loss = torch.mean((mu_hat - mu_tags_batch) ** 2)
        sigma_loss = torch.mean((Sigma_hat - sigma_tags_batch) ** 2)
        def hook_fn(grad):
            return
            print(grad)
        
        mu_hat.register_hook(hook_fn)
        optimal_values = envelope_layer.forward(L_hat, mu_hat)
        
        after_forward_memory = torch.cuda.memory_allocated()
        
        portfolio_loss = torch.mean(optimal_values)
        memory_stored += after_forward_memory - init_memory
        total_loss =  alpha*mu_loss  + beta*sigma_loss + gamma*portfolio_loss
        total_loss.backward()
        
        optimizer.step()
        epoch_loss += total_loss.item() * features_batch.size(0)
    
    epoch_loss /= len(dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.6f}')
# print(f"Average memory stored from forward pass: {(memory_stored/(len(dataloader) * num_epochs)) / 1e6:.2f} MB")  # Needs to run on CUDA, if not output is 0!

Epoch 1/5: 100%|██████████| 70/70 [00:13<00:00,  5.37it/s]


Epoch 1/5, Loss: -0.051563


Epoch 2/5: 100%|██████████| 70/70 [00:15<00:00,  4.48it/s]


Epoch 2/5, Loss: -0.132569


Epoch 3/5: 100%|██████████| 70/70 [00:17<00:00,  3.91it/s]


Epoch 3/5, Loss: -0.217064


Epoch 4/5: 100%|██████████| 70/70 [00:20<00:00,  3.46it/s]


Epoch 4/5, Loss: -0.300949


Epoch 5/5: 100%|██████████| 70/70 [00:21<00:00,  3.29it/s]

Epoch 5/5, Loss: -0.384180


In [8]:
# Inference Step
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA',  # Technology
    'PG', 'KO', 'PEP', 'WMT', 'COST',  # Consumer Goods
    'JNJ', 'PFE', 'MRK', 'ABT', 'AMGN',  # Healthcare
    'JPM', 'BAC', 'WFC', 'C', 'GS',  # Financials
    'XOM', 'CVX', 'COP', 'SLB', 'OXY',  # Energy
    'BA', 'CAT', 'UPS', 'DE', 'MMM',  # Industrials
    'DIS', 'TGT', 'MCD', 'LVMUY', 'NFLX',  # Consumer Services
    'LIN', 'NEM', 'CL', 'VMC', 'FCX',  # Materials
    'AMT', 'PLD', 'SPG', 'O', 'EQIX',  # Real Estate
    'NEE', 'DUK', 'XEL', 'SO', 'EXC'  # Utilities
]

rebalance_dates = [datetime(2024, month, 1) for month in range(1, 11)]

start_date = (rebalance_dates[0] - relativedelta(months=2)).strftime('%Y-%m-%d')
end_date = rebalance_dates[-1].strftime('%Y-%m-%d')
all_data = download_data(tickers, start_date=start_date, end_date=end_date)

portfolio_values_pred = [1.0]
portfolio_values_real = [1.0]
portfolio_values_equal = [1.0]
dates = [rebalance_dates[0]]
portfolio_returns_pred = []
portfolio_returns_real = []
portfolio_returns_equal = []

for i in range(len(rebalance_dates) - 1):
    rebalance_date = rebalance_dates[i]
    next_rebalance_date = rebalance_dates[i + 1]

    estimation_start_date = rebalance_date - relativedelta(months=2)
    estimation_end_date = rebalance_date
    holding_start_date = rebalance_date
    holding_end_date = next_rebalance_date

    returns_estimation = all_data.loc[estimation_start_date:estimation_end_date]
    returns_holding = all_data.loc[holding_start_date:holding_end_date]

    returns = returns_estimation.values
    returns_tensor = torch.tensor(returns, dtype=torch.float32)

    subseries_len = 28
    dataset = PortfolioDataset(returns_tensor, subseries_len=subseries_len)

    n_assets = len(tickers)
    input_dim = n_assets * subseries_len

    with torch.no_grad():
        features_batch, mu_tags_batch, sigma_tags_batch = dataset[0]
        features_batch = features_batch.unsqueeze(0)

        mu_hat, Sigma_hat, _ = model(features_batch)

        mu_pred = mu_hat[0].cpu().numpy()
        Sigma_pred = Sigma_hat[0].cpu().numpy()

        mu_real = np.mean(returns, axis=0)
        Sigma_real = np.cov(returns, rowvar=False)


    def mean_variance_optimization(mu, Sigma, risk_aversion=1.0):
        n = len(mu)
        w = cp.Variable(n)
        objective = cp.Maximize(mu @ w - risk_aversion / 2 * cp.quad_form(w, Sigma))
        constraints = [cp.sum(w) == 1, w >= 0]
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS)
        return w.value


    w_pred = mean_variance_optimization(mu_pred, Sigma_pred)
    w_real = mean_variance_optimization(mu_real, Sigma_real)

    w_pred = np.maximum(w_pred, 0)
    w_pred /= np.sum(w_pred)
    w_real = np.maximum(w_real, 0)
    w_real /= np.sum(w_real)
    w_equal = np.ones(n_assets) / n_assets

    returns_holding_values = returns_holding.values
    pred_returns = returns_holding_values @ w_pred
    real_returns = returns_holding_values @ w_real
    equal_returns = returns_holding_values @ w_equal

    portfolio_returns_pred.extend(pred_returns)
    portfolio_returns_real.extend(real_returns)
    portfolio_returns_equal.extend(equal_returns)

    cumulative_return_pred = np.prod(1 + pred_returns)
    cumulative_return_real = np.prod(1 + real_returns)
    cumulative_return_equal = np.prod(1 + equal_returns)

    new_value_pred = portfolio_values_pred[-1] * cumulative_return_pred
    new_value_real = portfolio_values_real[-1] * cumulative_return_real
    new_value_equal = portfolio_values_equal[-1] * cumulative_return_equal

    portfolio_values_pred.append(new_value_pred)
    portfolio_values_real.append(new_value_real)
    portfolio_values_equal.append(new_value_equal)
    dates.append(next_rebalance_date)


# Calculate drawdowns
def calculate_drawdown(portfolio_values):
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdown = (portfolio_values - cumulative_max) / cumulative_max
    return drawdown


drawdown_pred = calculate_drawdown(portfolio_values_pred)
drawdown_real = calculate_drawdown(portfolio_values_real)
drawdown_equal = calculate_drawdown(portfolio_values_equal)


# Calculate Sharpe ratios
def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    excess_returns = np.array(returns) - risk_free_rate
    return np.mean(excess_returns) / np.std(excess_returns)


sharpe_pred = calculate_sharpe_ratio(portfolio_returns_pred)
sharpe_real = calculate_sharpe_ratio(portfolio_returns_real)
sharpe_equal = calculate_sharpe_ratio(portfolio_returns_equal)

print(f"Sharpe Ratio (Predicted Portfolio): {sharpe_pred:.4f}")
print(f"Sharpe Ratio (Sample Portfolio): {sharpe_real:.4f}")
print(f"Sharpe Ratio (Equally Weighted Portfolio): {sharpe_equal:.4f}")

# Plot portfolio values
plt.figure(figsize=(12, 6))
plt.plot(dates, portfolio_values_pred, marker='o', label='Predicted MV Portfolio')
plt.plot(dates, portfolio_values_real, marker='o', label='Sample MV Portfolio')
plt.plot(dates, portfolio_values_equal, marker='o', label='Equally Weighted Portfolio')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Portfolio Backtest from 2024-01-01 to 2024-10-01')
plt.legend()
plt.grid(True)
plt.show()

# Plot drawdown as an area plot
plt.figure(figsize=(12, 6))
plt.fill_between(dates, drawdown_pred, color='blue', alpha=0.3, label='Predicted Portfolio Drawdown')
plt.fill_between(dates, drawdown_real, color='orange', alpha=0.3, label='Sample Portfolio Drawdown')
plt.fill_between(dates, drawdown_equal, color='green', alpha=0.3, label='Equally Weighted Portfolio Drawdown')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.title('Portfolio Drawdown from 2024-01-01 to 2024-10-01')
plt.legend()
plt.grid(True)
plt.show()

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (501 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        